In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv('marketing_campaign_data_messy.csv')
df

,Campaign_ID,Campaign_Name,Start_Date,End_Date,Channel,Impressions,Clicks,Spend,Conversions,Active,Clicks,Campaign_Tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24 00:00:00,2023-12-13,TikTok,16795,197,$102.82,20.0,Y,NaN,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06 00:00:00,2023-05-12,Facebook,1860,30,24.33,1.0,0,NaN,FA
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13 00:00:00,2023-12-20,Email,77820,843,1323.39,51.0,No,NaN,EM
3,CMP-00004,Q1_BlackFriday_CMP-00004,2023-10-30,2023-11-03,TikTok,55886,2019,2180.38,135.0,True,NaN,TI
4,CMP-00005,Q2_Winter_CMP-00005,2023-04-22 00:00:00,2023-04-23,Facebook,7265,169,252.44,30.0,Yes,NaN,FA
...,...,...,...,...,...,...,...,...,...,...,...,...
2015,CMP-00400,Q3_Summer_CMP-00400,2023-10-31 00:00:00,2023-11-13,TikTok,30592,586,$503.95,77.0,1,NaN,TI
2016,CMP-01255,Q4_Summer_CMP-01255,2023-09-01 00:00:00,2023-09-26,Google Ads,20097,897,1641.0,162.0,0,NaN,GO
2017,CMP-01050,Q2_Launch_CMP-01050,2023-02-09 00:00:00,2023-02-21,Instagram,33254,1117,883.82,214.0,0,NaN,IN
2018,CMP-01118,Q4_Winter_CMP-01118,2023-03-30 00:00:00,2023-04-27,Facebook,68728,2960,4198.5,591.0,Yes,NaN,FA


## HEADER CLEANING

In [13]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
print(df.columns.tolist())

['campaign_id', 'campaign_name', 'start_date', 'end_date', 'channel', 'impressions', 'clicks', 'spend', 'conversions', 'active', 'clicks', 'campaign_tag']


## Type conversion and currency cleaning

In [14]:
dirty_spend_mask = df['spend'].astype(str).str.contains(r'\$')
print(df.loc[dirty_spend_mask, ['campaign_id','spend']].head(3))

   campaign_id     spend
0    CMP-00001   $102.82
21   CMP-00022   $2428.4
22   CMP-00023  $4726.22


In [15]:
df['spend'] = df['spend'].astype(str).str.replace(r'[^\d.-]','', regex=True)
df['spend'] = pd.to_numeric(df['spend'])

In [16]:
print(df.loc[dirty_spend_mask, ['campaign_id','spend']].head(3))

   campaign_id    spend
0    CMP-00001   102.82
21   CMP-00022  2428.40
22   CMP-00023  4726.22


## Categorical Typos (Fuzzy Logic)

In [17]:
print(df['channel'].unique())

['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' 'E-mail' nan 'Gogle'
 'Tik_Tok' 'Facebok' 'Insta_gram']


In [18]:
cleanup_map = {
    'Facebok' : 'Facebook',
    'Insta_gram' : 'Instagram',
    'Tik_Tok': 'TikTok',
    'E-mail': 'Email',
    'Gogle': 'Google Ads',
    'N/A' : np.nan
}

df['channel'] = df['channel'].replace(cleanup_map)

In [19]:
print(df['channel'].unique())

['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' nan]


## Handling Mixed Booleans

In [20]:
df['active'].unique()

array(['Y', '0', 'No', 'True', 'Yes', '1', 'False'], dtype=object)

In [21]:
bool_map = {
    'Yes' : True,
    'Y' : True,
    '1' : True,
    1 : True,
    'False': False,
    'Yes': False,
    '0': False,
    0: False
}

df['active'] = df['active'].map(bool_map).fillna(False).astype(bool)

C:\Users\Dell\AppData\Local\Temp\ipykernel_15228\897310049.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['active'] = df['active'].map(bool_map).fillna(False).astype(bool)


In [22]:
df['active'].unique()

array([ True, False])

## Date Parsing

In [23]:
df['start_date'].dtype

dtype('O')

In [26]:
df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')

In [27]:
print(df['start_date'].dtype)

datetime64[ns]


## Logical Integrity (Clicks vs Impressions)

In [30]:
df = df.loc[:, ~df.columns.duplicated()]
df

,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,campaign_tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24,2023-12-13,TikTok,16795,197,102.82,20.0,True,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06,2023-05-12,Facebook,1860,30,24.33,1.0,False,FA
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13,2023-12-20,Email,77820,843,1323.39,51.0,False,EM
3,CMP-00004,Q1_BlackFriday_CMP-00004,NaT,2023-11-03,TikTok,55886,2019,2180.38,135.0,False,TI
4,CMP-00005,Q2_Winter_CMP-00005,2023-04-22,2023-04-23,Facebook,7265,169,252.44,30.0,False,FA
...,...,...,...,...,...,...,...,...,...,...,...
2015,CMP-00400,Q3_Summer_CMP-00400,2023-10-31,2023-11-13,TikTok,30592,586,503.95,77.0,True,TI
2016,CMP-01255,Q4_Summer_CMP-01255,2023-09-01,2023-09-26,Google Ads,20097,897,1641.00,162.0,False,GO
2017,CMP-01050,Q2_Launch_CMP-01050,2023-02-09,2023-02-21,Instagram,33254,1117,883.82,214.0,False,IN
2018,CMP-01118,Q4_Winter_CMP-01118,2023-03-30,2023-04-27,Facebook,68728,2960,4198.50,591.0,False,FA


In [31]:
impossible_rank = df['clicks'] > df['impressions']
print(df.loc[impossible_rank, ['campaign_id', 'impressions', 'clicks']].head(3))

Empty DataFrame
Columns: [campaign_id, impressions, clicks]
Index: []


## Logical Intergrity (Time Travel)

In [35]:
df[df['end_date'] < df['start_date']][['campaign_id', 'start_date', 'end_date']].head(3)

,campaign_id,start_date,end_date
23,CMP-00024,2023-05-06,2023-05-01
54,CMP-00055,2023-09-01,2023-08-27
71,CMP-00072,2023-02-01,2023-01-27


In [36]:
df.loc[df['end_date'] < df['start_date'], 'end_date'] = df.loc[df['end_date'] < df['start_date'], 'start_date'] + pd.Timedelta(days=30)

In [39]:
df[df['end_date'] < df['start_date']][['campaign_id', 'start_date', 'end_date']].head(3)

,campaign_id,start_date,end_date


## Handling outliers 

In [44]:
Q1 = df['spend'].quantile(0.25)
Q3 = df['spend'].quantile(0.75)

IQR = Q3 - Q1
upper_limit = Q3 + (1.5 * IQR)

outlier_mask = df['spend'] > upper_limit
print(df.loc[outlier_mask, ['campaign_id', 'spend']].head(3))

    campaign_id    spend
57    CMP-00058  5719.84
70    CMP-00071  5951.42
119   CMP-00120  7471.52


In [45]:
df.loc[outlier_mask, 'spend'] = upper_limit

In [46]:
print(df.loc[outlier_mask, ['campaign_id', 'spend']].head(3))

    campaign_id    spend
57    CMP-00058  5620.96
70    CMP-00071  5620.96
119   CMP-00120  5620.96


## String parsing (Feature Extraction)

In [47]:
print(df['campaign_name'].head(3))

0    Q4_Summer_CMP-00001
1    Q1_Launch_CMP-00002
2    Q3_Winter_CMP-00003
Name: campaign_name, dtype: object


In [48]:
df['season'] = df['campaign_name'].str.extract(r'Q\d_([^_]+)_')

In [50]:
print(df[['campaign_name','season']].head(3))

         campaign_name  season
0  Q4_Summer_CMP-00001  Summer
1  Q1_Launch_CMP-00002  Launch
2  Q3_Winter_CMP-00003  Winter
